In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from datetime import datetime
import requests
from selenium.webdriver.common.by import By

In [2]:
# Buka halaman web
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
driver.get('https://radartasik.id/category/jawa-barat/kota-tasikmalaya/')

In [8]:
try:
    # Inisialisasi counter
    counter = 0
    # Loop sampai counter mencapai 10
    while counter < 10:
        # Temukan tombol "Muat Selanjutnya" atau "Lihat Selengkapnya"
        load_more_button = driver.find_element(By.CLASS_NAME, 'view-more-button')
        if load_more_button:
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # Scroll ke tombol
            load_more_button.click()  # Klik tombol
            time.sleep(30)  # Tunggu konten dimuat
            counter += 1  # Tambah counter setiap kali tombol diklik
        else:
            print("Tombol 'Muat Selanjutnya' tidak ditemukan atau tidak ada lebih banyak konten untuk dimuat.")
            break
except Exception as e:
    print("Terjadi kesalahan:", e)

In [9]:
# Ambil sumber halaman setelah JavaScript dijalankan
html = driver.page_source

# Parsing HTML dengan BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Cari elemen-elemen yang diinginkan, misalnya berita highlight
articles = soup.find_all('article')

In [10]:
def custom_strptime_pr(date_str):
    # Mengonversi bulan ke angka
    bulan_dict = {
        "Januari": "01",
        "Februari": "02",
        "Maret": "03",
        "April": "04",
        "Mei": "05",
        "Juni": "06",
        "Juli": "07",
        "Agustus": "08",
        "September": "09",
        "Oktober": "10",
        "November": "11",
        "Desember": "12"
    }
    for bulan, angka in bulan_dict.items():
        date_str = date_str.replace(bulan, angka)

    # Mengonversi ke format datetime
    date_time_obj = datetime.strptime(date_str, "%d %m %Y")
    return date_time_obj

In [11]:
from selenium.common.exceptions import NoSuchElementException

def scrape_article_content(url):
    # Buka halaman berita
    driver.get(url)
    content = ''

    while True:
        # Tunggu hingga halaman dimuat
        time.sleep(10)
        
        # Ambil konten dari halaman saat ini
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # Mencari elemen konten berita yang sesuai
        article_content = soup.find('article', class_='post').find_all('p')  # Sesuaikan dengan class yang tepat
        # Mengambil teks dari elemen konten
        content_text = []
        for paragraph in article_content:
            content_text.append(paragraph.get_text(strip=True))
        
        # Menggabungkan semua teks konten
        article_text = ' '.join(content_text)
        print(article_text)

        # Cek apakah ada halaman berikutnya
        try:
            next_page_link = soup.find('a', attrs={'class': 'post-page-number'}, text='Laman berikutnya')
            prev_page_link = soup.find('a', attrs={'class': 'post-page-number'}, text='Laman sebelumnya')  # Sesuaikan dengan teks/link pagination yang sesuai
            if next_page_link:
                next_page_url = next_page_link['href']
                driver.get(next_page_url)  # Buka halaman berikutnya
            elif prev_page_link:
                break
            else:
                break  # Jika tidak ada halaman selanjutnya, keluar dari loop
        except NoSuchElementException:
            break  # Jika elemen tidak ditemukan, keluar dari loop

    return article_text

# Kemudian Anda bisa memanggil fungsi ini di dalam loop scraping utama Anda

In [12]:
# Buat list untuk menyimpan data
data = []
for article in articles:
    link = article.find('a')['href']
    judul = article.find('h2', attrs={'class': 'entry-title'}).string.strip()
    date_str = article.find('time', attrs={'class': 'entry-date'}).get_text(strip=True)
    # # Menggunakan fungsi custom_strptime_pr untuk mengkonversi tanggal
    date = custom_strptime_pr(date_str)
    print("Link:", link)
    print("Judul:", judul)
    print("Tanggal publish:", date_str)
    print(date)

    # get content
    article_content = scrape_article_content(link)
            
    # Menambahkan data ke list
    data.append({"url": link, "title": judul, "date": date, "content": article_content})

Link: https://radartasik.id/bela-palestina-besok-ribuan-massa-akan-padati-jalan-pusat-kota-tasikmalaya/
Judul: Bela Palestina, Besok Ribuan Massa Akan Padati Jalan Pusat Kota Tasikmalaya
Tanggal publish: 9 November 2023
2023-11-09 00:00:00


 TASIKMALAYA, RADARTASIK.ID– Ribuan massa di Kota Tasikmalaya akan turun ke jalan dan memadati jalanan pusat kota untuk melakukan aksi bela Palestina yang melibatkan berbagai elemen masyarakat, Jumat (10/11/2023).  Saat ini gerakan-gerakan moral dan sosial untuk Palestina terus bermunculan di nusantara. Media sosial pun sudah banyak dipenuhi pesan-pesan serupa di mana yang terjadi di jalur Gaza dinilai sebuah genosida.  Di Tasikmalaya juga aksi bela Palestina akan kembali terjadi, rencananya akan berlangsung di kawasan Taman Kota Tasikmalaya selepas Salat Jumat. Akan diisi oleh orasi-orasi, doa bersama dan dilanjutkan dengan long march menyusuri jalan di pusat Kota.  Baca juga :Mengancam UMKM dan Memicu Inflasi, Kalau Harga Cabai Rawit Tak Kunjung Terkendali  Rencananya, longmarch yang melibatkan ribuan massa itu akan dimulai dari Taman Kota ke Jalan HZ Mustofa, Jalan Tentara Pelajar, Jalan Otto Iskandar Dinata dan kembali ke kawasan Taman Kota atau Masjid Agung.  Koordinator aksi, Ase

/tmp/ipykernel_353/1515167307.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_link = soup.find('a', attrs={'class': 'post-page-number'}, text='Laman berikutnya')
/tmp/ipykernel_353/1515167307.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  prev_page_link = soup.find('a', attrs={'class': 'post-page-number'}, text='Laman sebelumnya')  # Sesuaikan dengan teks/link pagination yang sesuai


 TASIKMALAYA, RADARTASIK.ID– Janji Kadisdik Kota Tasikmalaya soal pelantikan kepala sekolah tampaknya sudah terpenuhi. Di mana Pj Wali Kota Tasikmalaya Dr Cheka Virgowansyah melantik para kepala sekolah baik SD, Rabu (8/11/2023).  102 Kepala Sekolah yang dilantik di aula Bale Kota Tasikmalaya itu meliputi 8 SMP dan 94 SD. Ada yang memang rotasi pindah penugasan, ada juga yang memang mendapat promosi.  Pada prosesi pelantikan tersebut, tampak hadir juga Sekda Kota Tasikmalaya Drs H Ivan Dicksan dan juga Kepala BKPSDM Gun Gun Pahlagunara.  Baca juga :Mengancam UMKM dan Memicu Inflasi, Kalau Harga Cabai Rawit Tak Kunjung Terkendali  Pj Wali Kota Tasikmalaya Dr Cheka Virgowansyah menyampaikan beberapa pesan kepada para kepala sekolah yang dilantik. Pertama yakni mengenai status sebagai pengajar di mana mereka tetaplah memiliki tugas sebagai guru. “Kepala sekolah adalah jabatan tambahan, tugas sebagai guru harus tetap dilaksanakan,” ungkapnya.  Mengingat tantangan dunia pendidikan saat ini 

In [13]:
import pandas as pd
# Membuat DataFrame dari list data
df = pd.DataFrame(data)
df

,url,title,date,content
0,https://radartasik.id/bela-palestina-besok-rib...,"Bela Palestina, Besok Ribuan Massa Akan Padati...",2023-11-09,"TASIKMALAYA, RADARTASIK.ID– Ribuan massa di K..."
1,https://radartasik.id/janji-kadisdik-kota-tasi...,"Janji Kadisdik Kota Tasikmalaya Terpenuhi, 102...",2023-11-09,"TASIKMALAYA, RADARTASIK.ID– Janji Kadisdik Ko..."
2,https://radartasik.id/sosok-ini-sukses-manfaat...,Sosok Ini Sukses Manfaatkan Lahan Tidur di Kot...,2023-11-09,"TASIKMALAYA, RADARTASIK.ID– Lahan tidur bisa ..."
3,https://radartasik.id/politisi-pdi-perjuangan-...,"Politisi PDI Perjuangan Ini Datangi Puskesmas,...",2023-11-09,"TASIKMALAYA, RADARTASIK.ID– Politisi PDI Perj..."
4,https://radartasik.id/pengawasan-korupsi-lemah...,"Pengawasan Korupsi Lemah, Pemkot Tasikmalaya D...",2023-11-09,"TASIKMALAYA, RADARTASIK.ID– Persoalan korupsi..."
...,...,...,...,...
215,https://radartasik.id/perampingan-dinas-di-kot...,Perampingan Dinas di Kota Tasikmalaya Akan Mem...,2023-10-16,"TASIKMALAYA, RADARTASIK.ID– Perampingan dinas..."
216,https://radartasik.id/2-pemuda-terlibat-pertar...,2 Pemuda Terlibat Pertarungan Jalanan Usai Mab...,2023-10-16,"TASIKMALAYA, RADARTASIK.ID– Dua pemuda terlib..."
217,https://radartasik.id/lapangan-alun-alun-dadah...,"Lapangan Alun-Alun Dadaha Diserbu Pengunjung, ...",2023-10-15,"TASIKMALAYA, RADARTASIK.ID– Setiap hal baru s..."
218,https://radartasik.id/akan-lebih-banyak-karyaw...,Akan Lebih Banyak Karyawan Asia Group Yang Dib...,2023-10-15,"TASIKMALAYA, RADARTASIK.ID– Sejumlah Karyawan..."


In [17]:
df['content'] = df['content'].str.slice(2)
df.head()

,url,title,date,content
0,https://radartasik.id/bela-palestina-besok-rib...,"Bela Palestina, Besok Ribuan Massa Akan Padati...",2023-11-09,Ribuan massa di Kota Tasikmalaya akan turun ke...
1,https://radartasik.id/janji-kadisdik-kota-tasi...,"Janji Kadisdik Kota Tasikmalaya Terpenuhi, 102...",2023-11-09,Janji Kadisdik Kota Tasikmalaya soal pelantika...
2,https://radartasik.id/sosok-ini-sukses-manfaat...,Sosok Ini Sukses Manfaatkan Lahan Tidur di Kot...,2023-11-09,Lahan tidur bisa jadi sumber cuan menjanjikan ...
3,https://radartasik.id/politisi-pdi-perjuangan-...,"Politisi PDI Perjuangan Ini Datangi Puskesmas,...",2023-11-09,Politisi PDI Perjuangan sekaligus Wakil Ketua ...
4,https://radartasik.id/pengawasan-korupsi-lemah...,"Pengawasan Korupsi Lemah, Pemkot Tasikmalaya D...",2023-11-09,Persoalan korupsi di Kota Tasikmalaya menjadi ...


In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/peminumyakult/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
import re, string, unicodedata  #modul regular expression
import nltk
from nltk import word_tokenize, sent_tokenize  #Paket ini membagi teks input menjadi kata-kata.,                                  
from nltk.corpus import stopwords

In [20]:
#preprocessing
def removeStopword(str):
    stop_words = set(stopwords.words('indonesian'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)
#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str
def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
       
    return str
def preprocessing(str):
    str = removeSentence(str)
    str = cleaning(str)
    str = removeStopword(str)
    
    return str

In [21]:
#test the code
sentences = ["dimana lokasi kuliner tasikmalaya yang murah","alamat tasik dimana sih, yang enak","s"]
for st in sentences:
    r = preprocessing(st)
    print(r)

dimana lokasi kuliner tasikmalaya murah
alamat tasik dimana sih enak



In [23]:
txt = df['content'].apply(preprocessing)

In [25]:
txt

0      ribuan massa kota tasikmalaya turun jalan mema...
1      janji kadisdik kota tasikmalaya pelantikan kep...
2      lahan tidur sumber cuan menjanjikan diolah sos...
3      politisi pdi perjuangan wakil ketua dprd kota ...
4      korupsi kota tasikmalaya sorotan kalangan maha...
                             ...                        
215    perampingan dinas digodok serius pemerintah ko...
216    pemuda terlibat keributan pertarungan jalanan ...
217    magnet kunjungan warga lapangan alun alun dada...
218    karyawan asia group hadiah umrah perusahaan di...
219    pelaku usaha berkembang penjualan onine menjag...
Name: content, Length: 220, dtype: object

In [26]:
txt.to_csv('data/radar-clean-data.csv', index=False)